Com os modelos feitos, agora quero uma combinação deles que eu possa usar pra ter um resultado melhor do que eu posso ter com um só.

Aqui nao vamos usar nada extremamente avançado de stacking, que sao metodos de ensemble muito mais complexo ate pelo tamanho dos dados a chance de funcionar nao e tao grande.

Como são dados relativamente simples, o maximo que vamos tentar aqui e uma média ponderada.


####  Até a parte do ensemble, tudo igual ao que fizemos aqui anteriormente.

In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

from lightgbm import LGBMClassifier

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("labels_curso - to_label_2.csv", index_col=0).dropna(subset=["y"])

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [4]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Limpeza da data

In [5]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Limpeza de Views

In [6]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [7]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [8]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [9]:
features.head()

,views,views_por_dia
0,28028,61.464912
394,1161,21.109091
393,141646,809.405714
392,325,21.666667
391,61,7.625000


In [10]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = (df_limpo['date'] >= "2019-04-01")

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((555, 2), (609, 2), (555,), (609,))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,3))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [12]:
title_bow_train.shape

(555, 1144)

In [13]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [14]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((555, 1146), (609, 1146))

# 4 RF

In [15]:
mdl_rf = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight="balanced", n_jobs=6)
mdl_rf.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=6, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
p_rf = mdl_rf.predict_proba(Xval_wtitle)[:, 1]

In [17]:
average_precision_score(yval, p_rf), roc_auc_score(yval, p_rf)

(0.2284201947891743, 0.6926785398360559)

# 5 LGBM

In [18]:
params = [0.08265121231498246, 7, 1, 0.7251351011494334, 0.07547006552546137, 839, 2, 3]
lr = params[0]
max_depth = params[1]
min_child_samples = params[2]
subsample = params[3]
colsample_bytree = params[4]
n_estimators = params[5]

min_df = params[6]
ngram_range = (1, params[7])

title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

mdl_lgbm = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                     min_child_samples=min_child_samples, subsample=subsample,
                     colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                     class_weight="balanced", n_jobs=6)
mdl_lgbm.fit(Xtrain_wtitle, ytrain)

p_lgbm = mdl_lgbm.predict_proba(Xval_wtitle)[:, 1]


/home/jarbasjr/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [19]:
average_precision_score(yval, p_lgbm), roc_auc_score(yval, p_lgbm)

(0.247808743128664, 0.6717874624049065)

# 7 Logistic Reg

In [20]:
from sklearn.pipeline import make_pipeline

##### Obs

A parte do "scaler" foi comentada.

Se a Reg Log entrar no ensemble, na solução final, é muito mais facil usar o método "make_pipeline". Pois a gente simplesmente passa os transformadores e modelos que a gente quer usar e ele vai aplicando sequencialmente. Então quando eu passo MaxAbsScaler() e LogisticRegressiion() pra função make_pipeline ele vai fazer a mesma coisa que estavamos fazendo com fit_transform quando o scaler tava fora da função.

In [21]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
#scaler = MaxAbsScaler()

#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())
#Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
#Xval_wtitle2 = scaler.transform(Xval_wtitle2)

lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(C=0.5, penalty='l2',n_jobs=6, random_state=0))
lr_pipeline.fit(Xtrain_wtitle2, ytrain)

Pipeline(memory=None,
         steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                ('logisticregression',
                 LogisticRegression(C=0.5, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=6, penalty='l2',
                                    random_state=0, solver='lbfgs', tol=0.0001,
                                    verbose=0, warm_start=False))],
         verbose=False)

In [22]:
p_lr = lr_pipeline.predict_proba(Xval_wtitle2)[:, 1]

In [23]:
average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)

(0.2150545436183453, 0.6870319042283423)

# 8 Ensemble

Anotando aqui todos os resultados e anotamos. Nós temos os scores de cada modelo. E a primeira coisa que Mario gosta de tentar é a média simples de todos os modelos.

##### Random Forest
(0.22228951304206077, 0.6914990859232175) RF  

##### LightGBM
(0.23779186526938, 0.6883293035324645) LGBM  

##### Logistic Regression
(0.2124987281512838, 0.6808987438815827) LR  


##### LGBM após mudança em ngram_range, colocando parametro igual do RF para facilitar o deploy
(0.247808743128664, 0.6717874624049065) LGBM ngram 1,3 - valor após mudança no ngram_range

In [106]:
# Média simples de todos os modelos. A gente nao bate a "average_precision" mas bate a "auc".
# Então, não é a solução que considerariamos final, mas ja é uma solução que não ta ruim.

p = (p_lr + p_rf + p_lgbm)/3
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.24346920623544438, 0.6936958188358789)

Uma coisa importante quando estamos fazendo ensemble é que os modelos sejam diferentes entre si.

E uma maneira de descobrir se os modelos estão encontrando soluções diferentes entre si é medir a correlação entre as previsões.

Pegamos as três previsões e calculamos a correlação de pearson simplesinha abaixo.

Olha que interessante, apesar de LGBM e RF serem dois modelos de ensemble de árvores, o LGBM só tem 0.55 de correlação com RF. Mario ja teve solução em competição que modelos com correlação de 0.97 entre eles e ainda assim melhoravam a solução. Então se a gente tem correlações tão baixas pra modelos com perfomances muito parecida, é um sinal bom que usando esses modelos juntos a gente vai ter um ganho.

após mudarmos o ngram_range do lgbm, a correlação entre lgbm e rf caiu mais ainda, o que é bem bom, nao perdemos perfomance.

In [107]:
pd.DataFrame({"LR": p_lr, "RF": p_rf, "LGBM": p_lgbm}).corr()

,LR,RF,LGBM
LR,1.000000,0.820536,0.460369
RF,0.820536,1.000000,0.486359
LGBM,0.460369,0.486359,1.000000


Depois de calcular a correlação, vamos tentar uma combinação somente random forest e lgbm pq são diferentes o bastante e tem uma perfomance muito boa comparada a reg log.

Eu poderia fazer uma busca exaustiva de coeficientes pros três, mas quero tentar só com esses dois até pra simplificar a parte do deploy.

In [110]:
# resultado que eu tenho não bate o lgbm sozinho na ap, mas bate qualquer modelo na 'auc'
# vamos especular os pesos, aumentando o  do lgbm, ficando 0.4/0.6, tivemos melhora, entao provavlmente estamos na direção certa
# apesar do auc ter diminuido um pouco quando comparao ao 0.5/0.5, ainda está melhor que as soluções individuais

# 0.3/0.7, essa variação na terceira casa é muito pequena, provavelmente é só ruido, mas a ap subiu bem com esses coef
# tentando 0.2/0.8, melhorando pouco, nao significativa.
# em 0.1/0.9 ja cai o desempenho, noassa maxima ta na região de 0.2/0.8
# apesar do 0.2/0.8 ter sido um pouco melhor, vamos ficar com 0.3/0.7, pq a melhora nao é tão maior que conveça ficar com 0.8 pro lgbm
# e tendo uma contribuição um pouco maior da random forest vamos ter um ensemble mais estavel, estará melhor distribuido o ensemble


p = 0.5*p_rf + 0.5*p_lgbm
average_precision_score(yval, p), roc_auc_score(yval, p)/*

(0.24316089022038515, 0.6923394468361149)

O LightGbm está com o TF-IDF diferente da random forest, na random forest o ngram_range é (1,3), e no lightgbm(1,5), e a gente poderia fazer o deploy disso numa boa, mas pra simplificar isso vamos tentar reduzir o tf-idf do lgbm e colocar os mesmos paramtros da random forest e ver se eu perco perfomance. Se ficar meio parecido a perfomance, eu prefiro ter só um tf-idf pra manter em produção do que os dois vectorizer pra ter um ganho muito pequeno.

Então vou lá na lista "params" do lgbm e troco o ultimo item de 5 para 3, que representa a ultima banda do parametro ngram_range do tf-idf.

Como resultado do lgbm individual, a ap melhorou e o auc caiu, pode ser que simplesmente ele não usou esse ngram_range pq nao encontrou na busca, se tivesse deixado um pouco mais de tempo ele buscando, ele poderia encontrar essa solução mas no nosso caso não encontrou e isso é perfeitamente normal.

Uma coisa que mario gosta de fazer, depois da tunagem do bayesiam optmization é tunar um pouqinho manualmente, pra tentar de fato extrai o maximo, nao será feito aqui. Mudar só um pouco para valores vizinhos.

(0.23117553771909904, 0.6964675355310491) - 0.5/0.5  
(0.23866391160240463, 0.6962906174441233) - 0.4/0.6  
(0.2449271153955049, 0.6967329126614378) - 0.3/0.7  
(0.24568903874837777, 0.6967329126614378) - 0.2/0.8  


##### Após mudança em ngram_range, colocando parametro igual do RF para facilitar o deploy

(0.24567146005469367, 0.6897151618800496) - 0.3/0.7 - lgbm ngram 1,3

(0.24809974466463763, 0.690865129445067) - 0.4/0.6 -  lgbm ngram 1,3


Com 0.5/0.5 a ap cai bastante e auc melhora bem, nesse caso valeria a pena fazer o deploy da reg log, pq a media simples entre esses 3 modelos está batendo uma media ponderada só entre esses dois. Apesar de não bater o lgbm com o ngram em 3 sozinho na ap.

Então agora é uma decisão subjetiva, eu posso fazer um deploy de uma solução bem mais complexa pq tem dois vectorizer e etc ou entao aceitar uma solução um pouco mais simples, média simples de dois modelos e ficar satisfeito, como estamos falando de um modelo simples, de uma primeira versão nao vamos perder tempo com isso e vamos colocar RF e LGBM em produção e o bom e que a gente já sabe que em algum momento a gente pode ir la e treinar mais esses modelos, Reg log e mudar o vectorizer de algum, mas num primeiro momento pra facilitar vamos usar só o RF e LGBM com apenas um vectorizer para o titulo

In [ ]:
# reduzir complexidade do vectorizer

# 9 Salvar modelos

Chegou a hora de salvar os modelos.

Mario gosta de usar joblib, pq e o recomendado pelo numpy e ele prefere do que o pickle, pq ele parece ser mais estavel para lidar com o tipo de array por trás desses modelos.

Salvando todos apesar de só usar lgbm e rf. tanto que comentamos reg log.

Isso vai salvar exatamente os modelos treinados pra gente em um objeto que a gente simplesmente vai usar a função jb.load() e vai carregar o nome desses arquivos e aplicar primeiro o title_vec e depois o light_gbm e a random forest nos nossos novos exemplos.

In [111]:
import joblib as jb

In [112]:
jb.dump(mdl_lgbm, "lgbm_20200208.pkl.z")
jb.dump(mdl_rf, "random_forest_20200208.pkl.z")
#jb.dump(lr_pipeline, "logistic_reg_20200208.pkl.z")
jb.dump(title_vec, "title_vectorizer_20200208.pkl.z")

['title_vectorizer_20200208.pkl.z']

AGORA QUE JA TEMOS MODELOS SALVOS, AGORA É PEGAR EXEMPLOS NOVOS E USAR EM PRODUÇÃO.

NÃO USAMOS DATASET DE TESTE AQUI, USAMOS O DATASET DE VALIDAÇÃO PRA TUNAR MODELOS E AJUSTAR COEFICIENTE DO ENSEMBLE. EM GERAL ISSO NAO E PROBLEMA. PODE SER QUE TENHA UM POUCO DE OVERFITING PQ ESTAMOS REUTILIZANDO PRA OTIMIZAR, MAS NAO SERÁ CASTASTŔOFICO, QUE VAMOS OLHAR O SCORE NA VALIDAÇÃO E FALAR "NOSSA É MUITO DIFERENTE EM PRODUÇÃO, DADOS NOVOS."

MARIO GOSTAVA MUITO DE SEPARAR EM TREINO, VALIDAÇÃO E TESTE MAS VIU NA PRATICA QUE NÃO HÁ TESTE QUE NEM O TESTE EM PRODUYÇÃO, ENTÃO E IMPORTANTE COLOCAR EM PRODUÇÃO O MAIS RAPIDO POSSIVEL. MAS PODEMOS FAZE MIL TESTES OFFLINE MAS QUANDO COLOCAMOS EM PRODUÇÃO TEM MUITOS DESAFIOS DIFERENTES, DISTRIBUIÇOES DIFERENTES E COISAS DIFERENTES PARA LIDAR. O QUE TEMOS QUE FAZER É COLOCAR EM PRODUÇÃO E SE O MODELO NAO TIVER FUNCIONANDO COMO ESPERADO, INVESTIGAR PARA ARRUMA-LO

